In [ ]:
!pip install TTS

In [ ]:
!pip install langdetect
!pip install pyttsx3
!pip install googletrans==3.1.0a0
!pip install SpeechRecognition
!pip install gtts
!pip install PdfReader
!pip install fitz
!pip install PyPDF2
!pip install frontend
!pip install TTS
from google.colab import files
from IPython.display import Audio, display
import torch
from TTS.api import TTS

In [ ]:
# Imports necessary libraries/modules for file upload, text translation, speech recognition, language detection, text-to-speech conversion, and PDF reading.
from google.colab import files
from googletrans import Translator
import speech_recognition as sr
from langdetect import detect
from gtts import gTTS
from IPython.display import Audio, display
from PyPDF2 import PdfReader
import os
import torch
from TTS.api import TTS
from pydub import AudioSegment
import pvfalcon
import numpy as np

# Function to get user confirmation for the correctness of translation.
def get_user_confirmation():
    print("Is the translation correct? (yes/no)")
    confirmation = input("Enter your response: ").strip().lower()
    return confirmation

# Function to get edited translation from the user.
def get_edited_translation(original_translation):
    print("Enter the corrected translation (Original: " + original_translation + "):")
    edited_translation = input("Corrected Translation: ").strip()
    return edited_translation

# Function to get the choice of input method (text or audio).
def get_input_choice():
    print("Select an option:")
    print("1: Text input")
    print("2: Upload recorded audio file")
    choice = input("Enter your choice (1, 2): ").strip()
    return choice

# Function to recognize speech from audio data using Google's Speech Recognition API.
def recognize_audio(audio_data, language=None):
    r = sr.Recognizer()
    try:
        if language:
            text = r.recognize_google(audio_data, language=language)
        else:
            detected_language = detect(r.recognize_google(audio_data))
            text = r.recognize_google(audio_data, language=detected_language)
            print("Detected language:", detected_language)
        return text, language if language else detected_language
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
        return None, None
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        return None, None

# Function to read text from a PDF file.
def read_text_from_pdf(file):
    pdf_text = ""
    try:
        with open(file, "rb") as f:
            reader = PdfReader(f)
            for page in reader.pages:
                pdf_text += page.extract_text()
    except Exception as e:
        print("Error reading PDF:", e)
    return pdf_text.strip()  # Strip extra whitespace

# Function to translate text to a specified language using Google Translate API.
def translate_text(text, dest_lang='en'):
    translator = Translator()
    try:
        translated = translator.translate(text, dest=dest_lang)
        if translated is not None:
            return translated.text
        else:
            print("Translation failed: No translation returned")
            return None
    except Exception as e:
        print("Translation failed:", e)
        return None

# Function to convert text to speech and play it.
def text_to_speech(text):
    tts = gTTS(text=text, lang='en')
    tts.save("translated_audio.mp3")
    display(Audio("translated_audio.mp3", autoplay=True))

# Function to process audio file using Falcon diarization and segmentation
def process_audio_with_falcon(audio_file_path):
    falcon = pvfalcon.create(access_key="ESj9DA04ueBKgpOflb2VGAQ4B0J+c9haXjJ4RTyg2F3hxKNZFrTJ6Q==")

    segments = falcon.process_file(audio_file_path)
    input_audio = AudioSegment.from_wav(audio_file_path)

    output_dir = "/content/segmented_audio"
    os.makedirs(output_dir, exist_ok=True)

    segment_paths = []
    for i, segment in enumerate(segments):
        start_ms = int(segment.start_sec * 1000)  # Convert start time to milliseconds
        end_ms = int(segment.end_sec * 1000)  # Convert end time to milliseconds
        speaker_tag = segment.speaker_tag

        segment_audio = input_audio[start_ms:end_ms]
        output_path = os.path.join(output_dir, f"segment_{i}_speaker_{speaker_tag}.wav")
        segment_audio.export(output_path, format="wav")

        segment_paths.append((output_path, segment.speaker_tag))

        print(f"Segment {i}: Speaker {speaker_tag}, Start: {segment.start_sec:.2f}s, End: {segment.end_sec:.2f}s saved to {output_path}")

    return segment_paths

# Function to merge audio segments
def merge_audio_segments(segment_paths, output_path):
    combined = AudioSegment.empty()
    for segment_path, _ in segment_paths:
        audio_segment = AudioSegment.from_wav(segment_path)
        combined += audio_segment
    combined.export(output_path, format="wav")
    return output_path

def main():
    choice = get_input_choice()
    if choice == '1':  # Text input
        print("Select method of text entry:")
        print("1: Type text in the textbox")
        print("2: Upload a PDF")
        text_entry_choice = input("Enter your choice (1, 2): ").strip()
        if text_entry_choice == '1':
            text = input("Enter text: ")
        elif text_entry_choice == '2':
            print("Upload the PDF:")
            pdf_file = files.upload()
            if not pdf_file:
                print("No file uploaded. Exiting.")
                return
            filename = list(pdf_file.keys())[0]
            text = read_text_from_pdf(filename)
            if not text:
                print("No text extracted from the PDF. Exiting.")
                return
            print("Text extracted from PDF:", text)
        else:
            print("Invalid choice. Please select 1 or 2.")
            return

        detected_language = detect(text)
        if detected_language:
            print("Input text language:", detected_language)
        else:
            print("Language detection failed. Please provide the input language code.")
            return
    elif choice == '2':  # Upload recorded audio file
        print("Upload the recorded audio file:")
        audio_file = files.upload()
        if not audio_file:
            print("No file uploaded. Exiting.")
            return
        filename = list(audio_file.keys())[0]

        # Process the audio with Falcon diarization and segmentation
        segments = process_audio_with_falcon(filename)

        output_dir = "/content/translated_audio"
        os.makedirs(output_dir, exist_ok=True)

        translated_segment_paths = []

        for segment_path, speaker_tag in segments:
            with sr.AudioFile(segment_path) as source:
                audio_data = sr.Recognizer().record(source)
            input_language = input(f"Enter the input language code for segment {segment_path} (e.g., 'hi' for Hindi): ").strip()
            text, detected_language = recognize_audio(audio_data, language=input_language)
            if not text:
                return
            print(f"Segment {segment_path}, Speaker {speaker_tag}: You said: {text}")

            target_language = input("Enter the language code for translation : ").strip()
            try:
                translated_text = translate_text(text, dest_lang=target_language)
                if translated_text:
                    print(f"Segment {segment_path}, Speaker {speaker_tag}: Translated text: {translated_text}")

                    device = "cuda" if torch.cuda.is_available() else "cpu"
                    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)
                    wav = tts.tts(text=translated_text, speaker_wav=segment_path, language=target_language)

                    # Display the generated audio before saving
                    display(Audio(wav, rate=22050))

                    # Normalize audio data and convert the numpy array to an AudioSegment
                    wav = np.array(wav)
                    wav = wav * (2**15 - 1) / np.max(np.abs(wav))  # Normalize to int16 range
                    wav = wav.astype(np.int16)
                    audio_segment = AudioSegment(
                        data=wav.tobytes(),
                        frame_rate=22050,
                        sample_width=2,  # 16-bit PCM
                        channels=1
                    )

                    # Save the translated audio segment
                    translated_audio_path = os.path.join(output_dir, f"translated_segment_{os.path.basename(segment_path)}.wav")
                    audio_segment.export(translated_audio_path, format="wav")

                    # Store translated audio path for merging
                    translated_segment_paths.append((translated_audio_path, speaker_tag))

                    # Display the translated audio segment for playback
                    display(Audio(translated_audio_path))
                    print(f"Translated audio segment saved to {translated_audio_path}")
                else:
                    print(f"Segment {segment_path}, Speaker {speaker_tag}: Translation failed.")
            except Exception as e:
                print(f"Error occurred while translating segment {segment_path}, Speaker {speaker_tag}: {e}")

        # Merge all translated segments into a single audio file
        if translated_segment_paths:
            merged_audio_path = "/content/merged_translated_audio.wav"
            merge_audio_segments(translated_segment_paths, merged_audio_path)
            print(f"All translated segments merged into {merged_audio_path}")

            # Display the merged audio for playback
            display(Audio(merged_audio_path))
    else:
        print("Invalid choice. Please select 1 or 2.")
        return

if __name__ == "__main__":
    main()
